# Get Image from given GIS data

+ Read shp file from ../Rawdata
+ Overlay layers according to color
+ Extract images by 128 by 128

In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
from PIL import Image

In [2]:
path = '../Rawdata/Total/'
EMD = path + 'Seoul_EMD.shp'
Parcel = path + 'Seoul_Parcel.shp'
Landuse = path + 'Seoul_Landuse.shp'
Building = path + 'Seoul_BULD.shp'
Road = path + 'Seoul_RW.shp'
Block = path + 'Seoul_Block.shp'

In [ ]:
#Sample_path = '../Rawdata/Sample/'
#EMD = Sample_path + 'Sample_EMD.shp'
#Block = Sample_path + 'Sample_Boundary.shp'
#Parcel = Sample_path + 'Sample_Parcel_2.shp'
#Landuse = Sample_path + 'Sample_Landuse.shp'
#Building = Sample_path + 'Sample_BULD.shp'
#Road = Sample_path + 'Sample_RW.shp'
#Block = Sample_path + 'Sample_Block.shp'


In [3]:
#Read File
EMD_gdf = gpd.read_file(EMD)
Parcel_gdf = gpd.read_file(Parcel)
#Landuse_gdf = gpd.read_file(Landuse)
Building_gdf = gpd.read_file(Building)
#Road_gdf = gpd.read_file(Road)
Block_gdf = gpd.read_file(Block)

EMD_gdf = EMD_gdf.to_crs(epsg = 5179)
Parcel_gdf = Parcel_gdf.to_crs(epsg = 5179)
#Landuse_gdf = Landuse_gdf.to_crs(epsg = 5179)
Building_gdf =  Building_gdf.to_crs(epsg = 5179)
#Road_gdf =  Road_gdf.to_crs(epsg = 5179)
Block_gdf = Block_gdf.to_crs(epsg = 5179)

In [4]:

ColorCode = {'UQA110': [245,245,7, '#F5F507'],
             'UQA111': [237, 248, 91, '#EDF85B'],
             'UQA112': [222, 237, 36, '#DEED24'],
             'UQA120': [248, 255, 105, '#F8FF69'],
             'UQA121': [247, 238, 48, '#F7EE30'],
             'UQA122': [252, 221, 43, '#FCDD2B'],
             'UQA123': [245, 203, 31, '#F5CB1F'],
             'UQA124': [245, 203, 31, '#F5CB1F'],
             'UQA130': [220, 168, 7, '#DCA807'],
             'UQA210': [211, 107, 66, '#D36B42'],
             'UQA220': [231, 153, 107, '#E7996B'],
             'UQA230': [241, 182, 140, '#F1B68C'],
             'UQA240': [253, 213, 172, '#FDD5AC'],
             'UQA330': [169, 169, 255, '#A9A9FF'],
             'UQA410': [157, 147, 79, '#9D934F'],
             'UQA420': [116, 215, 83, '#74D753'],
             'UQA430': [96, 202, 96, '#60CA60'],
             'UQA999': [169, 169, 169, '#A9A9A9']}
ColorCode_RGB = {k: v[:3] for k, v in ColorCode.items()}
ColorCode_16 = {k: v[-1] for k, v in ColorCode.items()}
print(ColorCode_16)

{'UQA110': '#F5F507', 'UQA111': '#EDF85B', 'UQA112': '#DEED24', 'UQA120': '#F8FF69', 'UQA121': '#F7EE30', 'UQA122': '#FCDD2B', 'UQA123': '#F5CB1F', 'UQA124': '#F5CB1F', 'UQA130': '#DCA807', 'UQA210': '#D36B42', 'UQA220': '#E7996B', 'UQA230': '#F1B68C', 'UQA240': '#FDD5AC', 'UQA330': '#A9A9FF', 'UQA410': '#9D934F', 'UQA420': '#74D753', 'UQA430': '#60CA60', 'UQA999': '#A9A9A9'}


In [5]:
# Add Color dictionary in Landuse
Landuse_gdf['color'] = Landuse_gdf['CODE']
Landuse_gdf = Landuse_gdf.replace({'color': ColorCode_16})

In [4]:

def trim_layer(boundary, target):
    '''
    Trim given target layer according to the boundary layer
    Input: GeoDataFrame, GeoDataFrame / Output: GeoDataFrame
    '''
    trimmed_layer = gpd.overlay(boundary, target)
    return trimmed_layer

#trimmed_layer = trim_layer(Block_gdf, Road_gdf)
#trimmed_layer.plot()


In [5]:
# Get trimmed layer by every district
def get_districts(boundary, target):
    '''
    Trim Every layers by Dong
    Input: GeoDataFrame, GeoDataFrame / Output: List ofUQ GeoDataFrame
    '''
    total = len(boundary)
    trimmed_layers = []
    for k, v in boundary.iterrows():
        p1 = v.geometry
        g = gpd.GeoSeries([p1])
        gdf = gpd.GeoDataFrame(crs = 'EPSG:5179', geometry = g)
        temp = trim_layer(gdf, target)
        trimmed_layers.append(temp)
        if k % 100 == 0 and k >= 100:
            print('{}/{}'.format(k, total))
    return trimmed_layers
#trimmed_LUs = get_districts(Block_gdf, Landuse_gdf)
#trimmed_RWs = get_districts(Block_gdf, Road_gdf)
#trimmed_PCLs = get_districts(Block_gdf, Parcel_gdf)
#rimmed_BULDs = get_districts(Block_gdf, Building_gdf)
#trimmed_BLKs = get_districts(EMD_gdf, Block_gdf)

In [6]:
EMD_gdf = EMD_gdf[:100]
EMD_BLKs = get_districts(EMD_gdf, Block_gdf)
#EMD_LUs = get_districts(EMD_gdffrom200, Landuse_gdf)
#EMD_RWs = get_districts(EMD_gdffrom200, Road_gdf)
EMD_PCLs = get_districts(EMD_gdf, Parcel_gdf)
EMD_BULDs = get_districts(EMD_gdf, Building_gdf)



In [7]:
#LUs4 = EMD_LUs

BLKs3 = EMD_BLKs

#RWs4 = EMD_RWs

PCLs3 = EMD_PCLs

BULDs3 = EMD_BULDs

In [ ]:
n = len(BLKs4)
print(len(LUs4),len(RWs4),len(PCLs4),len(BULDs4))
print(n)

In [ ]:
trimmed_LUs = []
trimmed_RWs = []
trimmed_PCLs = []
trimmed_BULDs = []
n = len(BLKs3)
#print(len(LUs4),len(RWs4),len(PCLs4),len(BULDs4))
for k, v in enumerate(BLKs3):
    #trimmed_LUs.append(get_districts(v, LUs4[k]))
    #trimmed_RWs.append(get_districts(v, RWs4[k]))
    trimmed_PCLs.append(get_districts(v, PCLs3[k]))
    trimmed_BULDs.append(get_districts(v, BULDs3[k]))
    
    if k % 20 == 0:
        print('{}/{}'.format(k, n))
    
print(len(trimmed_LUs))

In [11]:
#print(len(LUs3),len(RWs4),len(PCLs4),len(BULDs4))
print(len(trimmed_LUs), len(trimmed_RWs), len(trimmed_PCLs), len(trimmed_BULDs))

0 0 100 100


In [11]:
def flatten(t):
    return [item for sublist in t for item in sublist]

trimmed_LUs4 = flatten(trimmed_LUs)
trimmed_RWs4 = flatten(trimmed_RWs)
trimmed_PCLs4 = flatten(trimmed_PCLs)
trimmed_BULDs4 = flatten(trimmed_BULDs)
trimmed_BLKs4 = flatten(BLKs3)

print(len(trimmed_LUs4), len(trimmed_RWs4), len(trimmed_PCLs4), len(trimmed_BULDs4), len(trimmed_BLKs4))


0 0 5191 5191 334


In [ ]:
trimmed_BLKs1 = flatten(BLKs1)

In [ ]:
print(len(trimmed_BULDs1))

In [ ]:
# Give Color code on land use
def useColorcode(target_list):
    '''
    Apply color code on land use gdf to export img
    Input: List of List of GeoDataFrame  / Output: png
    '''
    LUs, RWs, PCLs = target_list
    n = len(LUs)
    print(n)
    my_dpi = 96
    for i in range(n):
        fig, ax = plt.subplots(figsize = (512/my_dpi, 512/my_dpi), dpi = my_dpi)
        ax.set_axis_off()
        ax.set_aspect('equal')
        LUs[i].plot(ax = ax, color = LUs[i]['color'])
        PCLs[i].plot(ax = ax, facecolor = 'none', edgecolor = 'black', lw = 0.1)
        #BLKs[i].plot(ax = ax, facecolor = 'none', edgecolor = 'white', lw = 0.2)
        RWs[i].plot(ax = ax, color = 'gray')
        
        #plt.show()
        fig_name = './blks4/sample_{}'.format(str(i))
        
        fig.savefig(fig_name)
    return 0
useColorcode([trimmed_LUs4, trimmed_RWs4, trimmed_PCLs4])

In [ ]:
if len(trimmed_BULDs2[6]):
    trimmed_BULDs2[0].fillna(0, inplace = True)
trimmed_BULDs2[6]

In [12]:
# Join building height to parcel
def parcel_height(parcels, buildings):
    '''
    Join average building height (floor) by matching parcel
    Input: List of GeoDataFrame, List of GeoDataFrame / Output: List of GeoDataFrame
    '''
    parcel_Hs = []
    for i in range(len(parcels)):
        parcel = parcels[i]
        building = buildings[i]
        building_h = building[['GRO_FLO_CO', 'geometry']]
        parcel_h = gpd.sjoin(parcel, building_h, how = 'left')
        parcel_meanH = parcel_h.groupby('PNU').mean()
        parcel_H = parcel.merge(parcel_meanH, on = 'PNU')
        try:
            parcel_H['GRO_FLO_CO'].fillna(0, inplace = True)
        except: pass
        parcel_Hs.append(parcel_H)
    return parcel_Hs

PCL_Hs = parcel_height(trimmed_PCLs4, trimmed_BULDs4)
print(len(PCL_Hs))
#print(PCL_Hs[0]['GRO_FLO_CO'])


5191


In [ ]:
# Parcel to color
h_colorcode = [(25/255, 0, 0), (50/255, 0, 0), (75/255, 0, 0), (100/255, 0, 0), (125/255, 0, 0),
               (150/255, 0, 0), (175/255, 0, 0), (200/255, 0, 0), (225/255, 0, 0), (250/255, 0, 0)]
plt.close()
def parcel_Hcolor(parcels):
    '''
    Give RGB gradation according to the parcel's height
    Input: List of GeoDataFrame / Output: List of imgs
    '''
    for i in range(len(parcels)):
        parcel = parcels[i]
        try:
            parcel['color'] = pd.qcut(parcel['GRO_FLO_CO'], bins = 10, labels = h_colorcode, include_lowest = True)
        except: pass
        #parcel['color'].replace(0, (0, 1,1), inplace = True)
        my_dpi = 96
        fig, ax = plt.subplots(figsize = (512/my_dpi, 512/my_dpi), dpi = my_dpi)
        ax.set_axis_off()
        try: parcel.plot(ax = ax, color = parcel['color'], edgecolor = 'None')
        except: parcel.plot(ax = ax, color = '#FFFFFF', edgecolor = 'None')
        fig_name = './new_bld4/sample_building_{}'.format(str(i))
        fig.savefig(fig_name)
    return parcel
parcel_Hcolor(PCL_Hs)

In [ ]:
# Color to height by pixel


In [ ]:
# Give Style
plt.ioff()
EMD_gdf.plot()
plt.savefig('./img/EMD.png')
plt.close()

In [ ]:
EMD_gdf.plot()

In [3]:
temp = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
cut = [[:3], [3:6], [6:]]


0
3
6
9
